In [88]:
one_time_run = True

In [86]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
import codecs
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
%matplotlib inline
pal = sns.color_palette()

from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

from keras.preprocessing.text import Tokenizer


In [3]:
#Paths
QUORA_DATA_DIR = "/opt/datasets/quora/"
TRAIN_CSV = QUORA_DATA_DIR+'train.csv'
TEST_CSV = QUORA_DATA_DIR+'test.csv'

GLOVE_DATA_DIR="/opt/datasets/glove/"
glove_840B_300d = GLOVE_DATA_DIR+'glove.840B.300d.txt'
GLOVE_DATA_FILE= glove_840B_300d

In [4]:
EMBEDDING_DIM = 300
MAX_SEQUENCE_LENGTH = 45
MAX_NB_WORDS = 200000
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.01

# Data Analysis

In [31]:
df_train = pd.read_csv(TRAIN_CSV)
df_test = pd.read_csv(TEST_CSV)

#Train Data
train_feature_1_string = pd.Series(df_train['question1'].tolist()).astype(str)
train_feature_2_string = pd.Series(df_train['question2'].tolist()).astype(str)
target = pd.Series(df_train['is_duplicate'].tolist())
all_train_qs = train_feature_1_string + train_feature_2_string

#Test Data
test_feature_1_string = pd.Series(df_test['question1'].tolist()).astype(str)
test_feature_2_string = pd.Series(df_test['question2'].tolist()).astype(str)
all_test_qs = test_feature_1_string + test_feature_2_string

all_qs = all_train_qs + all_test_qs

print(train_qs.tolist()[:10])

df_train.head()

['What is the step by step guide to invest in share market in india?', 'What is the story of Kohinoor (Koh-i-Noor) Diamond?', 'How can I increase the speed of my internet connection while using a VPN?', 'Why am I mentally very lonely? How can I solve it?', 'Which one dissolve in water quikly sugar, salt, methane and carbon di oxide?', 'Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?', 'Should I buy tiago?', 'How can I be a good geologist?', 'When do you use シ instead of し?', 'Motorola (company): Can I hack my Charter Motorolla DCX3400?']


,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


# Text Analysis

In [21]:
# plt.figure(figsize=(12, 5))
# plt.hist(dt_all_qids.value_counts(), bins=50)
# plt.yscale('log', nonposy='clip')
# plt.title('Log-Histogram of question appearance counts')
# plt.xlabel('Number of occurences of question')
# plt.ylabel('Number of questions')
# print()

In [32]:
total_ques_pairs = len(train)
print('Total number of question pairs for training: {}'.format(total_ques_pairs))

duplicate_ques_pairs = round(df_train['is_duplicate'].mean()*100, 2)
print('Duplicate pairs: {}%'.format(duplicate_ques_pairs))

unique_qids = len(np.unique(train_qs))
print('Total number of questions in the training data: {}'.format(unique_qids))

print('Number of questions that appear multiple times: {}'.format(np.sum(all_qids.value_counts() > 1)))

print("Total number of questions in Quora dataset: {}".format(len(all_qs)))

Total number of question pairs for training: 404290
Duplicate pairs: 36.92%
Total number of questions in the training data: 537362
Number of questions that appear multiple times: 111780
Total number of questions in Quora dataset: 2345796


In [23]:

# dist_train = train_qs.apply(len)
# dist_test = test_qs.apply(len)

# plt.figure(figsize=(15, 10))
# plt.hist(dist_train, bins=200, range=[0, 200], color=pal[2], normed=True, label='train')
# plt.hist(dist_test, bins=200, range=[0, 200], color=pal[1], normed=True, alpha=0.5, label='test')
# plt.title('Normalised histogram of character count in questions', fontsize=15)
# plt.legend()
# plt.xlabel('Number of characters', fontsize=15)
# plt.ylabel('Probability', fontsize=15)

# print('mean-train {:.2f} std-train {:.2f} mean-test {:.2f} std-test {:.2f} max-train {:.2f} max-test {:.2f} max-train{:.2f}'.format(dist_train.mean(), 
#                           dist_train.std(), dist_test.mean(), dist_test.std(), dist_train.max(), dist_test.max(), dist_train.max()))

In [24]:
# from wordcloud import WordCloud
# cloud = WordCloud(width=1440, height=1080).generate(" ".join(train_qs.astype(str)))
# plt.figure(figsize=(20, 15))
# plt.imshow(cloud)
# plt.axis('off')

In [26]:
%%time
if(one_time_run):
    print('Indexing word vectors.')
    embeddings_index = {}
    f = codecs.open(GLOVE_DATA_FILE, encoding='utf-8')
    for line in f:
        values = line.split(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    print('Found %s word vectors.' % len(embeddings_index))
else:
    print('Skipped to save some time!')
  

Indexing word vectors.
Found 2196017 word vectors.
CPU times: user 4min 22s, sys: 2.87 s, total: 4min 24s
Wall time: 4min 24s


In [75]:
one_time_run = False  

In [18]:
# embeddings_index['best']

In [27]:
# %%time
# all_questions_text = train_feature_1_text + train_feature_2_text + test_feature_1_text + test_feature_2_text
# all_questions_text = list(filter(lambda q: type(q) == str, all_questions_text))

In [28]:
# ques_lengths = list(map(lambda q: len(str(q).split(' ')) if(type(q) is str) else 0, all_questions_text))
# #Beware the questions has nan and numbers

# print(max(ques_lengths))   

In [ ]:
def getTokenizeModel(list_of_all_string, max_number_words):
    tokenizer = Tokenizer(nb_words = max_number_words)
    tokenizer.fit_on_texts(dt_all_questions_text)

In [33]:
# %time
# tokenizer = Tokenizer(nb_words=dt_MAX_NB_WORDS)
# tokenizer.fit_on_texts(all_questions_text)
# train_sequences_1 = dt_tokenizer.texts_to_sequences(train_feature_1_text)
# train_sequences_2 = dt_tokenizer.texts_to_sequences(train_feature_2_text)
# word_index = tokenizer.word_index
# print('Found %s unique tokens.' % len(word_index))

# dt_test_sequences_1 = tokenizer.texts_to_sequences(test_feature_1_text)
# dt_test_sequences_2 = tokenizer.texts_to_sequences(test_feature_2_text)

# data_1 = pad_sequences(train_sequences_1, maxlen=MAX_SEQUENCE_LENGTH)
# data_2 = pad_sequences(train_sequences_2, maxlen=MAX_SEQUENCE_LENGTH)
# labels = np.array(labels)
# print('Shape of data tensor:', data_1.shape)
# print('Shape of label tensor:', labels.shape)

# test_data_1 = pad_sequences(test_sequences_1, maxlen=MAX_SEQUENCE_LENGTH)
# test_data_2 = pad_sequences(test_sequences_2, maxlen=MAX_SEQUENCE_LENGTH)
# test_labels = np.array(test_labels)
# del test_sequences_1
# del test_sequences_2
# del train_sequences_1
# del train_sequences_2
# import gc
# gc.collect()

In [97]:
l = ["just say a word","of some length"]
ll = list(map(lambda l: l.split(" "), l))
seq_length = 5
ll
e = {"just": [0,0,0,0,1],
     "say" : [0,0,0,1,1],
     "a" : [0,1,1,1,0],
     "word" : [1,0,0,1,0],
     "of" : [0,1,0,1,0],
     "some" : [0,1,0,1,0],
     "length" : [0,0,1,1,0],
     '<PAD>': [0,0,0,0,0]
    }

In [105]:
def list_of_words_padding(list_of_words, seq_length=5, append=True):
    """Pads/slices given bag of words for specified length."""
    if len(list_of_words) == seq_length:
        return list_of_words
    if len(list_of_words) > seq_length:
        return list_of_words[:seq_length]
    
    tmp = ['<PAD>' for i in range(seq_length - len(list_of_words))]
    if append:
        return list_of_words + tmp
    return  tmp + list_of_words

def list_of_words_embed_transformer(embed_dict, list_of_words):
    """Transform list of words to list of embed_dict values."""
    return map(lambda x:embed_dict(x), list_of_words)

def bag_of_list_of_words_embed_transformer(embed_dict, bag_list_of_words):
    """Transform bag of list of words to bag of list of embed_dict values."""
    return map(list_of_words_embed_transformer, bag_list_of_words)

def layer1_padding(list_of_words):
    return list_of_words_padding(list_of_words=list_of_words, seq_length=7, append=True)


print(list(map(lambda x: list_of_words_padding(x, 2, True), ll)))
print(list(map(lambda x: list_of_words_padding(x, 2, False), ll)))
print(list(map(lambda x: layer1_padding(x), ll)))

print()

[['just', 'say'], ['of', 'some']]
[['just', 'say'], ['of', 'some']]


TypeError: bag_of_list_of_words_embed_transformer() missing 1 required positional argument: 'bag_list_of_words'

In [100]:
def get_feature(embed_dict, list_of_strings, max_seq_length, embed_dim):
    features = np.zeros((len(list_of_strings), max_seq_length, embed_dim), dtype=float)
    list_of_bag_of_words = map(lambda l: list_of_words_padding(list_of_words=l.split(),
                                                                seq_length=max_seq_length,
                                                                append=True),
                                list_of_strings)
    return list_of_bag_of_words

    
list(get_feature(e, l, 7, 5))    

[['just', 'say', 'a', 'word', '', '', ''],
 ['of', 'some', 'length', '', '', '', '']]

# Build the Graph

In [70]:
lstm_size = 256
lstm_layers = 1
batch_size = 250
learning_rate = 0.001

In [ ]:
# Create the graph object
graph = tf.Graph()
# Add nodes to the graph
with graph.as_default():
    inputs_ = tf.placeholder(tf.float32, [None, None, None], name='inputs') #[Number of ques, Seq/Ques Length, Embed Dims]
    labels_ = tf.placeholder(tf.float32, [None, None], name='labels')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')

http://suriyadeepan.github.io/2016-06-28-easy-seq2seq/

https://www.kaggle.com/c/word2vec-nlp-tutorial/details/part-1-for-beginners-bag-of-words

https://github.com/yuhaozhang/sentence-convnet

In [83]:
500 / 60


8.333333333333334